In [17]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sb

# General utils
from tqdm import tqdm
from os import path

from aicsshparam import shtools
import pyvista as pv

def reconstruct_mesh(coeffs,lmax=5):
    coeffs = coeffs.to_dict()
    coeffs = {'_'.join(k.split('_')[1:3]):v for k,v in coeffs.items()}
    # Convert to matrix
    mat = np.zeros((2, lmax + 1, lmax + 1), dtype=np.float32)
    for L in range(lmax):
        for M in range(L + 1):
            for cid, C in enumerate(["C", "S"]):
                key = f"shcoeffs_L{L}M{M}{C}"
                if key in coeffs.keys():
                    mat[cid, L, M] = coeffs[key]
                else:
                    mat[cid,L,M] = 0
    mesh = shtools.get_even_reconstruction_from_coeffs(mat)
    return mesh


In [31]:
dataset_dir = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Mesa et al/Lineage models/Dataset pickles'
figure_dir = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Mesa et al/Lineage models/Random_forest_birth'
all_df = pd.read_pickle(path.join(dataset_dir,f'all_df.pkl'))
basals = pd.read_pickle(path.join(dataset_dir,f'basals.pkl'))

# Load the raw coeffs
dirnames = {'R1':'/Users/xies/OneDrive - Stanford/Skin/Mesa et al/W-R1/',
            'R2':'/Users/xies/OneDrive - Stanford/Skin/Mesa et al/W-R2/'}
regions = {name:pd.read_csv(path.join(dirname,'Mastodon/single_timepoints.csv')) for name,dirname in dirnames.items()}

cyto_coef_cols = [f for f in regions['R1'].columns if 'cyto_shcoeffs' in f and 'surface' not in f]
nuc_coef_cols = [f for f in regions['R1'].columns if 'nuc_shcoeffs' in f and 'surface' not in f]

coefs_raw = []
for name,region in regions.items():
    _cyto = region[cyto_coef_cols].copy()
    _cyto['Frame'] = region['Frame']
    _cyto['TrackID'] = name + '_' + region['TrackID'].astype(str)

    _nuc = region[nuc_coef_cols]
    _nuc['Frame'] = region['Frame']
    _nuc['TrackID'] = name + '_' + region['TrackID'].astype(str)
    
    coefs_raw.append(pd.merge(_cyto,_nuc, how='outer',left_on = ['Frame','TrackID'],right_on=['Frame','TrackID']))

coefs_raw = pd.concat(coefs_raw).set_index(['Frame','TrackID'],drop=True)

/var/folders/zp/5k1kvm9j0t19cvq8h628kr8h0000gn/T/ipykernel_94492/852862690.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _nuc['Frame'] = region['Frame']
/var/folders/zp/5k1kvm9j0t19cvq8h628kr8h0000gn/T/ipykernel_94492/852862690.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _nuc['TrackID'] = name + '_' + region['TrackID'].astype(str)
/var/folders/zp/5k1kvm9j0t19cvq8h628kr8h0000gn/T/ipykernel_94492/852862690.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

# Visualize the PCAs

In [43]:
def plot_cells_side_by_side(cells):
    num_cells = len(cells)
    if num_cells > 5:
        error
    pl = pv.Plotter(shape=(1,num_cells))
    
    for i in range(num_cells):
        pl.subplot(0,i)
        pl.add_mesh(pv.wrap(cells[i]['nuc']),color='r', opacity=0.5)
        pl.add_mesh(pv.wrap(cells[i]['cyto']),color='b', opacity=0.4)
        pl.add_axes()
    pl.link_views()
    pl.view_isometric()
    pl.show()

def reconstruct_mesh_from_averaged_coeffs_(index,coeffs,cyto_col,nuc_col):
    mean_coeffs = coeffs.loc[index].mean()
    mean_cyto = mean_coeffs[cyto_col]
    mean_nuc = mean_coeffs[nuc_col]

    avg_cell = {'cyto':reconstruct_mesh(mean_cyto)[0],
                'nuc':reconstruct_mesh(mean_nuc)[0],
               }
    return avg_cell

def get_sorted_cell_indexes(df,feature):
    return df.droplevel(axis=1,level=1).sort_values(feature).index

In [47]:
## Get top/bottom by feature
feature = 'nuc_shcoeff_PC0'
high = reconstruct_mesh_from_averaged_coeffs_(get_sorted_cell_indexes(basals,feature)[-10:],
                                   coefs_raw, cyto_coef_cols, nuc_coef_cols)
low = reconstruct_mesh_from_averaged_coeffs_(get_sorted_cell_indexes(basals,feature)[:10],
                                   coefs_raw, cyto_coef_cols, nuc_coef_cols)

feature = 'nuc_shcoeff_PC9'
high2 = reconstruct_mesh_from_averaged_coeffs_(get_sorted_cell_indexes(basals,feature)[-10:],
                                   coefs_raw, cyto_coef_cols, nuc_coef_cols)
low2 = reconstruct_mesh_from_averaged_coeffs_(get_sorted_cell_indexes(basals,feature)[:10],
                                   coefs_raw, cyto_coef_cols, nuc_coef_cols)

plot_cells_side_by_side([high,low,high2,low2])

Widget(value='<iframe src="http://localhost:60299/index.html?ui=P_0x30e107620_18&reconnect=auto" class="pyvist…

In [51]:
basals.loc[get_sorted_cell_indexes(basals,'nuc_shcoeff_PC0')[0:10]]

,Name,Nuclear volume,Nuclear solidity,Z,Y,X,Nuclear height,Cell volume,Z-cyto,Y-cyto,X-cyto,...,Diff from Min adjac nuc_shcoeff_PC6 at 1 frame prior,Num neighbor division 1 frame prior,Num neighbor delamination 1 frame prior,Num daughter differentiated,At least one daughter differentiated,Both daughters differentiated,Region,Cell cycle duration,G1 duration,SG2 duration
,Metadata,Measurement,Measurement,Measurement,Measurement,Measurement,Measurement,Measurement,Measurement,Measurement,Measurement,...,Measurement,Measurement,Measurement,Meta,Meta,Meta,,Measurement,Measurement,Measurement
Frame,TrackID,,,,,,,,,,,,,,,,,,,,,
13,R1_309,149.1875,0.830550,52.605362,54.251780,89.913699,-9.0,389.3750,52.654575,54.406621,89.735594,...,0.697096,0.0,2.0,NaN,NaN,NaN,R1,NaN,NaN,NaN
14,R1_272,146.8750,0.885456,47.139149,22.316915,83.685426,-10.0,442.5625,47.563762,22.208163,83.728922,...,1.513620,0.0,0.0,NaN,NaN,NaN,R1,NaN,NaN,NaN
13,R1_852,134.0000,0.870130,50.958022,34.972132,10.470849,-10.0,363.3750,50.856209,35.136266,10.579076,...,-3.042970,0.0,0.0,NaN,NaN,NaN,R1,NaN,NaN,NaN
12,R1_791,130.6875,0.836735,53.728360,91.335485,85.367288,-9.0,415.0000,53.286295,91.563893,85.631739,...,0.436129,0.0,1.0,NaN,NaN,NaN,R1,NaN,NaN,NaN
11,R1_669,168.2500,0.881178,51.237741,14.479662,55.968889,-11.0,439.3125,51.076825,14.386897,55.851188,...,0.648609,6.0,1.0,NaN,NaN,NaN,R1,NaN,60.0,NaN
13,R1_863,130.5625,0.818574,52.614648,47.616084,35.544639,-11.0,401.4375,51.852872,47.912502,35.540908,...,-1.714572,0.0,0.0,NaN,NaN,NaN,R1,NaN,NaN,NaN
2,R2_774,159.8125,0.888773,55.943684,50.710207,89.012124,-11.0,345.1875,55.850081,50.771456,89.176308,...,-0.715758,1.0,0.0,NaN,NaN,NaN,R2,NaN,NaN,NaN
13,R1_304,136.0000,0.840479,49.739430,50.338810,83.612477,-10.0,446.9375,49.369319,50.645469,83.552860,...,2.434199,0.0,2.0,NaN,NaN,NaN,R1,NaN,NaN,NaN


In [53]:
basals

Metadata       Measurement
Frame TrackID             
0     R1_2        1.982310
      R1_5       -1.557531
      R1_11       1.795796
      R1_19       3.913186
      R1_24      -0.146453
...                    ...
14    R2_1134          NaN
      R2_1144          NaN
      R2_1149          NaN
      R2_1155          NaN
      R2_1156          NaN

[8216 rows x 1 columns]